In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

binary_data_path = '/content/drive/MyDrive/NLP_Workspace/Data/BERT_data/'

df = pd.read_csv(binary_data_path + "16class_BERT.csv")
df = df.dropna(how='any',axis=0)

In [3]:
df.sample(10)

,label,text
15412,1,telling good friend zdenek jirotka impressive ...
184226,14,subjectivity fine support end argument feel ob...
79497,6,ca throught whneis gon na stop want head want ...
109716,8,well abnormal levels energy wah -_-
114434,9,cast member disney world unattainable fun
175815,14,yes makes say sorry really confusing say want ...
165082,13,even little kid told people like colors think ...
162462,13,like going doctor sick paranoid find something...
2656,0,ohhh lol quite funny
101597,8,better understanding really know kind hoping s...


In [4]:
DOWNSAMPLE = 50000
df_balanced = df.sample(DOWNSAMPLE)
df_balanced['label'].value_counts()

11    3187
10    3181
14    3179
5     3178
15    3159
6     3155
4     3138
2     3125
13    3119
0     3104
12    3102
7     3094
8     3092
1     3084
3     3075
9     3028
Name: label, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_balanced, test_size=0.15)

In [6]:
train.head(3)

,label,text
30337,2,sorry still idea lol follow questions find fix...
191212,15,istp waste kind time
19966,1,yeah happens time either hey remind someone


In [7]:
test.head(3)

,label,text
29102,2,really know anything entps girl know seems cau...
107559,8,hi everyone infj found myers briggs personalit...
21676,1,read thread getting argument arkigos monemi fl...


In [9]:
ourpur_dir = binary_data_path + "16class_BERT/"
train.to_csv(ourpur_dir+"16class_train_data.csv", index=False)
test.to_csv(ourpur_dir+"16class_test_data.csv", index=False)

In [10]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 28.4 MB/s 
     |████████████████████████████████| 132 kB 69.7 MB/s 
     |████████████████████████████████| 182 kB 71.2 MB/s 
     |████████████████████████████████| 212 kB 68.0 MB/s 
     |████████████████████████████████| 127 kB 73.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [11]:
data_files = {"train": ourpur_dir+"16class_train_data.csv", "test": ourpur_dir+"16class_test_data.csv"}

from datasets import load_dataset
squad_it_dataset = load_dataset("csv", data_files=data_files, sep=',')

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5d40a23e37f27220/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 42500
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 7500
    })
})

In [13]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 28.6 MB/s 
     |████████████████████████████████| 7.6 MB 56.5 MB/s 


In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

In [16]:
TRAIN_NUM = int(DOWNSAMPLE * 0.85)
TEST_NUM = int(DOWNSAMPLE * 0.15)
TEST_NUM

7500

In [17]:
train_dataset = squad_it_dataset["train"].shuffle(seed=42).select([i for i in list(range(TRAIN_NUM))])
test_dataset = squad_it_dataset["test"].shuffle(seed=42).select([i for i in list(range(TEST_NUM))])

tokenized_train = train_dataset.map(preprocess_function)
tokenized_test = test_dataset.map(preprocess_function)

  0%|          | 0/42500 [00:00<?, ?ex/s]

  0%|          | 0/7500 [00:00<?, ?ex/s]

In [18]:
tokenized_train[0]

{'label': 6,
 'text': 'really rule-breaker mostly think rules reason followed changed become obviously wrong unless rule speed limit',
 'input_ids': [101,
  2428,
  3627,
  1011,
  24733,
  3262,
  2228,
  3513,
  3114,
  2628,
  2904,
  2468,
  5525,
  3308,
  4983,
  3627,
  3177,
  5787,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [20]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=16)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

In [27]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average='micro')["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [29]:
training_args = TrainingArguments(
    
    output_dir= ourpur_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 42500
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 13285
  Number of trainable parameters = 66965776
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster

Step,Training Loss
500,2.771800
1000,2.697200
1500,2.643400
2000,2.628800
2500,2.612200
3000,2.535400
3500,2.491800
4000,2.477700
4500,2.458700
5000,2.438600


Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT/checkpoint-500
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT/checkpoint-1000
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive

TrainOutput(global_step=13285, training_loss=2.2263002578683664, metrics={'train_runtime': 993.1253, 'train_samples_per_second': 213.971, 'train_steps_per_second': 13.377, 'total_flos': 1589412515052672.0, 'train_loss': 2.2263002578683664, 'epoch': 5.0})

In [30]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7500
  Batch size = 16


{'eval_loss': 2.494119882583618,
 'eval_accuracy': 0.26986666666666664,
 'eval_f1': 0.26986666666666664,
 'eval_runtime': 10.7558,
 'eval_samples_per_second': 697.299,
 'eval_steps_per_second': 43.604}